# downsampling - gridSearch

- it takes longer time. 
- better accuracy(minor in this case)
- labelEncoding is reqired
- grid search
- https://github.com/WillKoehrsen/Machine-Learning-Projects/blob/master/random_forest_explained/Improving%20Random%20Forest%20Part%202.ipynb

In [111]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing
from IPython.display import display   

df = pd.read_csv('../input/imputed_data.csv')
df.iloc[:,0]
df_s = df.iloc[:,1:]
df_s['weekday']=df_s['weekday'].astype('object')
df_s.head()

,sp_lim,road,weather,light,crash,weekday,report_type,num_units,device,damage,street_direction,binned_speed
0,25,NO_DEFECT,CLEAR,DAYLIGHT,0,7,no_report,1.0,NO,"$501 - $1,500",S,M
1,30,NO_DEFECT,CLEAR,DAYLIGHT,0,6,no_report,1.0,NO,"$501 - $1,500",N,M
2,30,NO_DEFECT,CLEAR,DAYLIGHT,0,7,no_report,1.0,NO,"$501 - $1,500",N,M
3,30,DEFECTIVE,CLEAR,DAYLIGHT,0,4,no_report,2.0,WORKING,"OVER $1,500",S,M
4,30,NO_DEFECT,CLEAR,DAYLIGHT,0,6,ON SCENE,2.0,WORKING,"OVER $1,500",S,M


In [112]:
# downsampling
from sklearn.utils import resample

df_majority = df_s[df_s.crash == 0]
df_minority = df_s[df_s.crash == 1]
print(df_majority.shape)
print(df_minority.shape)

(241189, 12)
(80815, 12)


In [113]:
df_majority_downsampled = resample(df_majority, 
                                 replace=False,    # sample without replacement
                                 n_samples=80815,     # to match minority class
                                 random_state=123) # reproducible results
 
# Combine minority class with downsampled majority class
df_downsampled = pd.concat([df_majority_downsampled, df_minority])
 
# Display new class counts
df_downsampled.crash.value_counts()
# now we see it is same sample size

1    80815
0    80815
Name: crash, dtype: int64

In [114]:
import pprint
# convert each categorical feature using one-hot encoding

obj_df = df_downsampled.select_dtypes(include = ['object'])
int_df = df_downsampled.select_dtypes(include = ('number'))

# onehot encoding
onehot_obj = pd.get_dummies(obj_df, drop_first=True) # dummy trap(one variable can be induced)
print(onehot_obj.columns)

# concat and # reset the index of design_df
design_df = pd.concat([onehot_obj,int_df], axis=1).reset_index().drop(columns='index',axis=1)


Index(['road_NO_DEFECT', 'weather_NO_CLEAR', 'light_DAYLIGHT', 'light_LIGHTED',
       'light_POOR_LIGHT', 'weekday_2', 'weekday_3', 'weekday_4', 'weekday_5',
       'weekday_6', 'weekday_7', 'report_type_ON SCENE',
       'report_type_no_report', 'device_NO', 'device_WORKING',
       'damage_$501 - $1,500', 'damage_OVER $1,500', 'street_direction_N',
       'street_direction_S', 'street_direction_W', 'binned_speed_L',
       'binned_speed_M'],
      dtype='object')


In [115]:
# order change to make it easy
design_df['target'] = design_df['crash']
design_df = design_df.drop(columns='crash',axis=1)


In [63]:
print(design_df.target.value_counts())


1    80815
0    80815
Name: target, dtype: int64


0         0
1         0
2         0
3         0
4         0
         ..
161625    1
161626    1
161627    1
161628    1
161629    1
Name: target, Length: 161630, dtype: int64

In [64]:
# feature selecting
X = design_df.iloc[:,:-1]
y = design_df.target
X

,road_NO_DEFECT,weather_NO_CLEAR,light_DAYLIGHT,light_LIGHTED,light_POOR_LIGHT,weekday_2,weekday_3,weekday_4,weekday_5,weekday_6,...,device_WORKING,"damage_$501 - $1,500","damage_OVER $1,500",street_direction_N,street_direction_S,street_direction_W,binned_speed_L,binned_speed_M,sp_lim,num_units
0,1,0,1,0,0,0,0,0,0,0,...,0,1,0,0,1,0,0,1,30,2.0
1,1,0,1,0,0,0,0,0,0,1,...,0,0,1,1,0,0,1,0,15,2.0
2,1,0,1,0,0,0,0,0,1,0,...,0,1,0,0,0,1,0,1,30,2.0
3,1,0,1,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,1,30,2.0
4,1,0,1,0,0,0,0,0,0,1,...,1,0,1,0,1,0,0,1,35,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
161625,1,0,1,0,0,0,0,0,1,0,...,1,0,1,1,0,0,0,1,30,2.0
161626,1,0,1,0,0,0,0,1,0,0,...,0,1,0,1,0,0,0,1,30,3.0
161627,0,1,1,0,0,0,1,0,0,0,...,0,0,1,0,0,1,0,1,30,3.0
161628,0,0,1,0,0,0,0,0,0,0,...,0,1,0,0,1,0,0,1,30,2.0


In [65]:
# # 설명 변수 데이터를 정규화(normalization)
from sklearn import preprocessing
X = preprocessing.StandardScaler().fit(X).transform(X)

# train data 와 test data로 구분(7:3 비율)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=10) 

In [66]:
print('Training Features Shape:', X_train.shape)
print('Training Class Shape:', y_train.shape)
print('Testing Features Shape:', X_test.shape)
print('Testing Labels Shape:', y_test.shape)

Training Features Shape: (113141, 24)
Training Class Shape: (113141,)
Testing Features Shape: (48489, 24)
Testing Labels Shape: (48489,)


In [67]:
y_train.shape

(113141,)

### Examine the Default Random Forest to Determine Parameters
- We will use these parameters as a starting point. I relied on the sklearn random forest documentation to determine which features to change and the available options.

In [70]:
from sklearn.ensemble import RandomForestClassifier

base_clf = RandomForestClassifier(random_state = 2002, oob_score=True)

from pprint import pprint
# Look at parameters used by our current forest
print('Parameters currently in use:\n')
pprint(base_clf.get_params()) # print nicely


Parameters currently in use:

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': True,
 'random_state': 2002,
 'verbose': 0,
 'warm_start': False}


This is a base model. We will compare this model with models below: random search and gridSearch.

### Random search with Cross Validation
We will set the hyperparameters randomly.

In [75]:
from sklearn.model_selection import RandomizedSearchCV

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 100, stop = 500, num = 5)]

# Number of features to consider at every split
max_features = ['auto', 'sqrt']

# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 50, num = 5)]
max_depth.append(None)

# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]

# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]

# # Method of selecting samples for training each tree
# bootstrap = [True, False]

# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               
               }

pprint(random_grid)

{'max_depth': [10, 20, 30, 40, 50, None],
 'max_features': ['auto', 'sqrt'],
 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [2, 5, 10],
 'n_estimators': [100, 200, 300, 400, 500]}


In [90]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
base_clf = RandomForestClassifier(random_state = 2002,oob_score=True)
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
randomCV = RandomizedSearchCV(estimator=base_clf, param_distributions=random_grid,
                              n_iter = 100, scoring='neg_mean_absolute_error', 
                              cv = 3, verbose=2, random_state=2002, n_jobs=-1,
                              return_train_score=True)

# Fit the random search model
randomCV.fit(X_train, y_train)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:   48.1s
[Parallel(n_jobs=-1)]: Done 130 tasks      | elapsed:  8.6min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed: 18.1min finished


RandomizedSearchCV(cv=3, error_score=nan,
                   estimator=RandomForestClassifier(bootstrap=True,
                                                    ccp_alpha=0.0,
                                                    class_weight=None,
                                                    criterion='gini',
                                                    max_depth=None,
                                                    max_features='auto',
                                                    max_leaf_nodes=None,
                                                    max_samples=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_samples_split=2,
                                                    min_weight_fraction_leaf=0.0,
               

In [77]:
randomCV.best_params_

{'n_estimators': 300,
 'min_samples_split': 10,
 'min_samples_leaf': 1,
 'max_features': 'auto',
 'max_depth': 10}

## Evaluation Function

### EV default model

In [91]:
from sklearn import metrics as mt
def evaluate(model, X_test, y_test):
    yhat = model.predict(X_test)
    oob_score = model.oob_score_
    report= mt.classification_report(y_test, yhat) 
    print('***** Model Performance. *****')
    print('- Out of Bag eror: {:0.4f}.'.format(oob_score))
    print('- y_test values:',y_test.values[0:10])
    print('- yhat   values:',yhat[0:10],'\n')
    print('<Accuracy> \n{}'.format(report))
    
    return report

In [92]:
base_clf.fit(X_train, y_train)
base_accuracy = evaluate(base_clf, X_test, y_test)

***** Model Performance. *****
- Out of Bag eror: 0.7691.
- y_test values: [1 0 1 0 1 0 0 1 0 0]
- yhat   values: [1 1 1 1 1 0 1 1 1 0] 

<Accuracy> 
              precision    recall  f1-score   support

           0       0.78      0.75      0.77     24301
           1       0.76      0.79      0.77     24188

    accuracy                           0.77     48489
   macro avg       0.77      0.77      0.77     48489
weighted avg       0.77      0.77      0.77     48489



In [93]:
randomCV.best_estimator_

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=10, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=10,
                       min_weight_fraction_leaf=0.0, n_estimators=300,
                       n_jobs=None, oob_score=True, random_state=2002,
                       verbose=0, warm_start=False)

### EV best random search model

In [94]:
best_random = randomCV.best_estimator_
random_accuracy = evaluate(best_random, X_test, y_test)

***** Model Performance. *****
- Out of Bag eror: 0.7872.
- y_test values: [1 0 1 0 1 0 0 1 0 0]
- yhat   values: [1 1 0 1 1 0 1 1 1 0] 

<Accuracy> 
              precision    recall  f1-score   support

           0       0.80      0.76      0.78     24301
           1       0.77      0.81      0.79     24188

    accuracy                           0.79     48489
   macro avg       0.79      0.79      0.79     48489
weighted avg       0.79      0.79      0.79     48489



### grid search
- We can now perform grid search building on the result from the random search. We will test a range of hyperparameters around the best values returned by random search.

In [95]:
from sklearn.model_selection import GridSearchCV

# Create the parameter grid based on the results of random search 
param_grid = {
    'bootstrap': [True],
    'max_depth': [60, 70, 80, 90],
    'max_features': [2, 3],
    'min_samples_leaf': [3, 4, 5],
    'min_samples_split': [8, 10, 12],
    'n_estimators': [100, 300, 500, 1000],
    'oob_score' : [True]
}

# Create a base model
best_clf = RandomForestClassifier(random_state = 2002)

# Instantiate the grid search model
grid_search = GridSearchCV(estimator = best_clf, param_grid = param_grid, 
                          cv = 3, n_jobs = -1, verbose = 2, return_train_score=True)

In [96]:
# Fit the grid search to the data
grid_search.fit(X_train, y_train)

Fitting 3 folds for each of 288 candidates, totalling 864 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:   53.9s
[Parallel(n_jobs=-1)]: Done 130 tasks      | elapsed: 10.6min
C:\Users\galma\Anaconda3\lib\site-packages\joblib\externals\loky\process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=-1)]: Done 333 tasks      | elapsed: 27.6min
[Parallel(n_jobs=-1)]: Done 616 tasks      | elapsed: 51.4min
[Parallel(n_jobs=-1)]: Done 864 out of 864 | elapsed: 72.1min finished


GridSearchCV(cv=3, error_score=nan,
             estimator=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                              class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              max_samples=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators=100, n_jobs=None,
                                              oob_score=False,
                                              rando

In [97]:
grid_search.best_params_

{'bootstrap': True,
 'max_depth': 60,
 'max_features': 3,
 'min_samples_leaf': 5,
 'min_samples_split': 8,
 'n_estimators': 1000,
 'oob_score': True}

### EV the best model from grid search


In [98]:
best_grid = grid_search.best_estimator_
grid_accuracy = evaluate(best_grid, X_train, y_train)

***** Model Performance. *****
- Out of Bag eror: 0.7853.
- y_test values: [0 0 1 0 1 0 1 0 1 1]
- yhat   values: [0 0 1 1 1 1 1 0 1 1] 

<Accuracy> 
              precision    recall  f1-score   support

           0       0.81      0.78      0.80     56514
           1       0.79      0.82      0.80     56627

    accuracy                           0.80    113141
   macro avg       0.80      0.80      0.80    113141
weighted avg       0.80      0.80      0.80    113141



### Final model
- The final model from hyperparameter tuning is as follows.

In [100]:
final_model = grid_search.best_estimator_

print('***** Final Model Parameters: *****\n')
pprint(final_model.get_params())
print('\n')
grid_final_accuracy = evaluate(final_model, X_test, y_test)

***** Final Model Parameters: *****

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': 60,
 'max_features': 3,
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 5,
 'min_samples_split': 8,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 1000,
 'n_jobs': None,
 'oob_score': True,
 'random_state': 2002,
 'verbose': 0,
 'warm_start': False}


***** Model Performance. *****
- Out of Bag eror: 0.7853.
- y_test values: [1 0 1 0 1 0 0 1 0 0]
- yhat   values: [1 1 1 1 1 0 1 0 1 0] 

<Accuracy> 
              precision    recall  f1-score   support

           0       0.80      0.77      0.78     24301
           1       0.78      0.81      0.79     24188

    accuracy                           0.79     48489
   macro avg       0.79      0.79      0.79     48489
weighted avg       0.79      0.79      0.79     48489



## Comparison of All Improvement Techniques
- Finally, we can make comparisons between the improvement techniques to determine which ones were most effective.

In [101]:
# Time used for evaluating model run times
import time

In [102]:
# Evaluate run time and prediction accuracy
def evaluate_model(model, X_train, y_train, X_test, y_test):
    n_trees = model.get_params()['n_estimators']
    n_features = X_train.shape[1]
    
    # Train and predict 10 times to evaluate time and accuracy
    predictions = []
    run_times = []
    for _ in range(10):
        start_time = time.time()
        model.fit(X_train, y_train)
        predictions.append(model.predict(X_test))
    
        end_time = time.time()
        run_times.append(end_time - start_time)
    
    # Run time and predictions need to be averaged
    run_time = np.mean(run_times)
    predictions = np.mean(np.array(predictions), axis = 0)
    
    # Calculate performance metrics
    out_of_bag_score = model.oob_score_
    yhat= model.predict(X_test)
    accuracy = mt.accuracy_score(y_test,yhat)
    
    # Return results in a dictionary
    results = {'time': run_time, 'score': out_of_bag_score, 'accuracy': accuracy, 'n_trees': n_trees, 'n_features': n_features}
    
    return results

### base model
- 10sec: 0.6 means 6secs

In [104]:
base_results = evaluate_model(base_clf, X_train, y_train, X_test, y_test)
base_results['model']='base_model'
base_results

{'time': 7.371490049362182,
 'score': 0.7691111091469935,
 'accuracy': 0.7714739425436697,
 'n_trees': 100,
 'n_features': 24,
 'model': 'base_model'}

### random search model

In [105]:
random_results = evaluate_model(best_random, X_train, y_train, X_test, y_test)
random_results['model']='best_random'
random_results

{'time': 13.047905898094177,
 'score': 0.7871682237208439,
 'accuracy': 0.7876838045742334,
 'n_trees': 300,
 'n_features': 24,
 'model': 'best_random'}

### best Grid Search Best Model

In [106]:
best_grid_results = evaluate_model(best_grid, X_train, y_train, X_test, y_test)
best_grid_results['model']='best_grid'
best_grid_results

{'time': 54.15622870922088,
 'score': 0.7853386482353877,
 'accuracy': 0.7864257872919631,
 'n_trees': 1000,
 'n_features': 24,
 'model': 'best_grid'}

## Comparison

In [109]:
comparison = {'model': [base_results['model'], random_results['model'], best_grid_results['model'] ],
              'accuracy': [round(base_results['accuracy'], 3), round(random_results['accuracy'], 3),round(best_grid_results['accuracy'], 3) ],
              'score': [round(base_results['score'], 3), round(random_results['score'], 3), round(best_grid_results['score'], 3)],
              'n_features': [base_results['n_features'], random_results['n_features'], best_grid_results['n_features']],
              'n_trees': [base_results['n_trees'], int(random_results['n_trees']), int(best_grid_results['n_trees'])],
              'time': [round(base_results['time'], 4), round(random_results['time'], 4), round(best_grid_results['time'], 4)]}


In [110]:

comparison = pd.DataFrame.from_dict(comparison, orient = 'columns')
comparison[['model', 'accuracy', 'score', 'n_features', 'n_trees', 'time']]

,model,accuracy,score,n_features,n_trees,time
0,base_model,0.771,0.769,24,100,7.3715
1,best_random,0.788,0.787,24,300,13.0479
2,best_grid,0.786,0.785,24,1000,54.1562
